In [1]:
import pandas as pd
import numpy as np
import vaex
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_absolute_error,accuracy_score
from sklearn.preprocessing import LabelEncoder
import joblib

In [2]:
data =pd.read_csv('Doceree-HCP_Train.csv', encoding='latin-1')
test =pd.read_csv('Doceree-HCP-Test.csv', encoding='latin-1')

In [3]:
# Assuming 'data' is your pandas DataFrame containing string columns
# Initialize the LabelEncoder
label_encoder = LabelEncoder()
# Iterate over each column in the DataFrame
for column in data.columns:
    # Check if the column contains string data
    if data[column].dtype == 'object':
        # Encode the string data to numerical values
        data[column] = label_encoder.fit_transform(data[column])

# Now, your string columns are converted to numerical values
# You can proceed with applying logistic regression on the dataset

In [4]:
# The Same thing for test_dataset
for column in test.columns:
    if test[column].dtype == 'object':
        test[column] = label_encoder.fit_transform(test[column])

In [5]:
vaex_data=vaex.from_pandas(data)
vaex_test=vaex.from_pandas(test)

In [6]:
vaex_data.export_hdf5('Doceree-HCP_Train.hdf5')
vaex_test.export_hdf5('Doceree-HCP-Test.hdf5')

In [7]:
vaex_df=vaex.open('Doceree-HCP_Train.hdf5')
# Iterate over each column in the DataFrame
for column in vaex_df.columns:
    # Fill missing values with 0 for the current column
    vaex_df[column].fillna(0, inplace=True)

In [8]:
vaex_test=vaex.open('Doceree-HCP-Test.hdf5')
# Iterate over each column in the DataFrame
for column in vaex_test.columns:
    # Fill missing values with 0 for the current column
    vaex_test[column].fillna(0, inplace=True)

In [9]:
vaex_df = vaex_df.shuffle(random_state=42)  # Shuffle the DataFrame with a fixed random state
df_train,df_test = vaex_df.ml.train_test_split(test_size=0.2)  # Split into train and test sets

c:\Users\kunal\miniconda3\lib\site-packages\vaex\ml\__init__.py:28: UserWarning: Make sure the DataFrame is shuffled
  warnings.warn('Make sure the DataFrame is shuffled')


In [10]:
features=['DEVICETYPE', 'PLATFORM_ID', 'BIDREQUESTIP', 'USERPLATFORMUID', 'USERCITY', 'USERZIPCODE', 'USERAGENT', 'PLATFORMTYPE', 'CHANNELTYPE', 'URL', 'KEYWORDS']
target='IS_HCP'
txo_features = ['DEVICETYPE', 'BIDREQUESTIP', 'USERPLATFORMUID', 'USERCITY', 'USERZIPCODE', 'USERAGENT', 'PLATFORMTYPE', 'CHANNELTYPE', 'URL', 'KEYWORDS']
txo_target=['TAXONOMY']

In [11]:
# Create the RandomForestRegressor model
model = RandomForestRegressor()

In [12]:
# Fill missing values with a specific value (e.g., 0)
df_train = df_train.fillna(0)
df_test = df_test.fillna(0)

In [13]:
# Extract the features and target arrays from the DataFrames
X_train = df_train[features].values
y_train = df_train[target].values
txo_train = y_train #to use in futuer

X_test = df_test[features].values
y_test = df_test[target].values
txo_test = y_test #to use in futuer

In [14]:
# Fit the model
model.fit(X_train, y_train)

RandomForestRegressor()

In [15]:
# Set the threshold for classification
threshold = 0.5

# Make predictions
predictions = model.predict(X_test)

# Apply threshold for classification
class_predictions = np.where(predictions >= threshold, 1, 0)

In [16]:
# Calculate and print the evaluation metrics
r2 = r2_score(y_test, class_predictions)
mae = mean_absolute_error(y_test, class_predictions)

print("R-squared:", r2)
print("Mean Absolute Error:", mae)

R-squared: 0.9627679291430701
Mean Absolute Error: 0.007811471453021459


Txo

In [17]:
# Extract the features and target arrays from the DataFrames
X_train = df_train[features].values
y_train = df_train[txo_target].values

X_test = df_test[features].values
y_test = df_test[txo_target].values

In [18]:
X_train = X_train[txo_train == 1]
y_train = y_train[txo_train == 1]
y_train = y_train.ravel()

X_test =  X_test[txo_test == 1]
y_test =  y_test[txo_test == 1]
y_test = y_test.ravel()

In [19]:
# Creating model
txo_model = RandomForestRegressor()

In [20]:
# Fit model
txo_model.fit(X_train,y_train)

RandomForestRegressor()

In [21]:
# Model Evaluation
t_prediction = txo_model.predict(X_test)

In [22]:
# Calculate and print the evaluation metrics
r2 = r2_score(y_test, t_prediction)
mae = mean_absolute_error(y_test, t_prediction)

print("R-squared:", r2)
print("Mean Absolute Error:", mae)

R-squared: 0.6145756666152081
Mean Absolute Error: 22.17758321873893


In [23]:
# Save the trained model
joblib.dump(model, 'trained_model.pkl')

# # Load the trained model
# txo_model = joblib.load('trained_model.pkl')

# # Load the vaex DataFrame for prediction
# vaex_prediction = vaex.open('Doceree-HCP-Test.hdf5')

# # Convert vaex DataFrame to NumPy array
# np_prediction = vaex_prediction.to_pandas().values

# # Make predictions
# predictions = txo_model.predict(np_prediction)

# # Apply threshold for classification if needed
# threshold = 0.5
# class_predictions = np.where(predictions >= threshold, 1, 0)

# # Calculate and print the evaluation metrics
# r2 = r2_score(y_test, class_predictions)
# mae = mean_absolute_error(y_test, class_predictions)
# accuracy = accuracy_score(y_test, class_predictions)

# print("R-squared:", r2)
# print("Mean Absolute Error:", mae)
# print("Accuracy:", accuracy)

['trained_model.pkl']